In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
dataset = keras.datasets.cifar10

In [4]:
(X_train, y_train), (X_test, y_test) = dataset.load_data()

In [5]:
X_train.shape

(50000, 32, 32, 3)

In [6]:
X_train = X_train.reshape(len(X_train), 32, 32, 3)
X_test = X_test.reshape(len(X_test), 32, 32, 3)

In [25]:
from warnings import filters
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(32, 32, 3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
   
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [26]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [27]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="cifra10")

INFO:tensorflow:Reloading Oracle from existing project output/cifra10/oracle.json
INFO:tensorflow:Reloading Tuner from output/cifra10/tuner0.json


In [28]:
tuner_search.search(X_train,y_train,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [29]:
model = tuner_search.get_best_models(num_models=1)[0]

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 48)        27696     
                                                                 
 flatten (Flatten)           (None, 37632)             0         
                                                                 
 dense (Dense)               (None, 80)                3010640   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 3,040,938
Trainable params: 3,040,938
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1407/1407 [==============================] - 10s 5ms/step - loss: 1.4323 - accuracy: 0.4829 - val_loss: 1.6045 - val_accuracy: 0.4284
Epoch 5/10
1407/1407 [==============================] - 8s 5ms/step - loss: 1.1318 - accuracy: 0.5967 - val_loss: 1.8231 - val_accuracy: 0.3990
Epoch 6/10
1407/1407 [==============================] - 8s 5ms/step - loss: 0.8157 - accuracy: 0.7139 - val_loss: 2.1203 - val_accuracy: 0.4322
Epoch 7/10
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5347 - accuracy: 0.8175 - val_loss: 2.5340 - val_accuracy: 0.4258
Epoch 8/10
1407/1407 [==============================] - 8s 5ms/step - loss: 0.3623 - accuracy: 0.8782 - val_loss: 3.3929 - val_accuracy: 0.4382
Epoch 9/10
1407/1407 [==============================] - 8s 5ms/step - loss: 0.2789 - accuracy: 0.9118 - val_loss: 3.7183 - val_accuracy: 0.4410
Epoch 10/10
1407/1407 [==============================] - 8s 5ms/step - loss: 0.2276 - accuracy: 0.9291 - val_loss: 3.8748 - val_accurac

In [32]:
predicted = model.predict(X_test[:4])
predicted

array([[1.84585124e-01, 2.26665705e-07, 3.35387565e-04, 3.45332384e-01,
        5.26624732e-03, 1.59441024e-01, 9.32062630e-06, 1.24082409e-01,
        1.64666399e-01, 1.62814949e-02],
       [1.66481491e-02, 1.32158697e-02, 9.66567863e-13, 3.81154353e-12,
        1.38565147e-12, 2.49078376e-17, 5.44244337e-11, 9.89186831e-18,
        9.69770312e-01, 3.65660875e-04],
       [9.59447771e-02, 2.86205132e-02, 2.32639839e-03, 4.14774404e-05,
        1.15265069e-03, 8.29329656e-06, 7.51376792e-05, 1.02537786e-04,
        8.69419038e-01, 2.30913097e-03],
       [6.09986894e-02, 4.55344940e-04, 6.13141321e-02, 5.36195701e-04,
        8.96691869e-04, 2.74902486e-05, 8.15632869e-04, 1.31753322e-05,
        8.74919116e-01, 2.35236384e-05]], dtype=float32)

In [33]:
y_test[:4]

array([[3],
       [8],
       [8],
       [0]], dtype=uint8)

In [34]:
y_test = y_test.reshape(-1,)
y_test[:4]

array([3, 8, 8, 0], dtype=uint8)

In [35]:
y_classes = [np.argmax(element) for element in predicted]
y_classes[:4]

[3, 8, 8, 8]